Load dataset:

In [1]:
from HumanGenomeDataset.load_dataset import load_dataset

data_df = load_dataset('dna_protein_coding_sequences')
data_df.head(2)

c:\Users\giuli\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,ID,sequence,category
0,NM_001368254.1,GCTGAGCTGAGCTGGGGCGCAGCCGCCTGTCTGCACCGGCAGCACC...,NM
1,NM_001350977.1,TCCACAACTGAAACATCCACTTCTGAACACCATGTCCTACTACAGC...,NM


Start simulation:

In [2]:
from src.simulation import ProteinSinthesisProcess
SIM_TIME = 50

In [3]:
protein_synthesis_process = ProteinSinthesisProcess(dna_sequences_df=data_df, verbose=True)
protein_synthesis_process.run(simulation_time=SIM_TIME) # run the simulation

Simulation environment initialized
Simulation started:
Time 0.0000: DNA Sequence 0 requesting to start synthesis
Time 0.0000: DNA Sequence 0 synthesize started
Time 0.0000: DNA Sequence 0 start transcription process
Time 0.0000: DNA Sequence 0 contains 6 promoters
Time 0.0000: DNA Sequence 0 (mRNA sequence 0) start transcription process
Time 0.1113: DNA Sequence 1 requesting to start synthesis
Time 0.1113: DNA Sequence 1 synthesize started
Time 0.1113: DNA Sequence 1 start transcription process
Time 0.1113: DNA Sequence 1 contains 2 promoters
Time 0.1113: DNA Sequence 1 (mRNA sequence 0) start transcription process
Time 0.7526: DNA Sequence 2 requesting to start synthesis
Time 0.7526: DNA Sequence 2 synthesize started
Time 0.7526: DNA Sequence 2 start transcription process
Time 0.7526: DNA Sequence 2 contains 1 promoters
Time 0.7526: DNA Sequence 2 (mRNA sequence 0) start transcription process
Time 0.9057: DNA Sequence 3 requesting to start synthesis
Time 0.9057: DNA Sequence 3 synthes

c:\Users\giuli\anaconda3\lib\site-packages\Bio\Seq.py:2880: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


Time 4.3113: DNA Sequence 1 synthetis ended
Time 4.3113: DNA Sequence 8 synthesize started
Time 4.3113: DNA Sequence 8 start transcription process
Time 4.3113: DNA Sequence 8 contains 1 promoters
Time 4.3113: DNA Sequence 8 (mRNA sequence 0) start transcription process
Time 4.4676: DNA Sequence 10 requesting to start synthesis
Time 4.5613: DNA Sequence 2 (mRNA sequence 0) end translation process
Time 4.5613: DNA Sequence 2 end transcription and translation process
Time 4.5613: DNA Sequence 2 synthetis ended
Time 4.5613: DNA Sequence 9 synthesize started
Time 4.5613: DNA Sequence 9 start transcription process
Time 4.5613: DNA Sequence 9 contains 0 promoters
Time 4.5613: DNA Sequence 9 synthetis ended
Time 4.5613: DNA Sequence 10 synthesize started
Time 4.5613: DNA Sequence 10 start transcription process
Time 4.5613: DNA Sequence 10 contains 3 promoters
Time 4.5613: DNA Sequence 10 (mRNA sequence 0) start transcription process
Time 4.6613: DNA Sequence 3 (mRNA sequence 1) end translation

Results:

In [4]:
results_df = protein_synthesis_process.dna_sequences_df

In [5]:
print('Number of DNA sequence traslated:', results_df[results_df['mrna_sequences'].notna()].shape[0])
print('Number of DNA sequence not traslated:', results_df[results_df['protein_synthesized']==False].shape[0])

Number of DNA sequence traslated: 19
Number of DNA sequence not traslated: 18


In [6]:
results_df[results_df['mrna_sequences'].notna()][[
    'polypeptides_chains', 'number_of_proteins_synthesized']]

,polypeptides_chains,number_of_proteins_synthesized
6807,"[NH2-MSSSFIKRPSLIFRHNITTEV-COOH, NH2-MSSSFIKRP...",3
6832,[NH2-MSQTRHDLTVVVDRPGSYGRLWGVRLHPGCPTGVASPGVSR...,1
8597,"[NH2-MTAPSHLKASTVYP-COOH, NH2-MTAPSHLKASTVYP-C...",2
8729,[NH2-MFYSNFDTSFRISQGRQTSTSSLDRGVSHWCIKRLKRTTLK...,2
13480,[NH2-MRGRHASAPVVTTETSSSEPPYRVSDPLTRAPGRLRWPEMA...,1
21397,[NH2-MDVLDRVRPVPQGGGTRHGRGRSLGGTSHSYSSRCGCHRSR...,2
21454,"[NH2-MSIDPGTSTTSIL-COOH, NH2-MSIDPGTSTTSLL-COO...",4
38095,"[NH2-MQKPRWGLVYYRHKLVFRQSLY-COOH, NH2-MQKPRWGL...",2
49628,"[NH2-MPHSRKD-COOH, NH2-MPHSRKD-COOH, NH2-MPHSR...",4
69870,[NH2-MLGRTSLPGASGLPGPEGPPQVRGAPTSSNFPNICSPSCSF...,4


In [9]:
type(results_df['polypeptides_chains']) #FIXME

pandas.core.series.Series

In [8]:
# see if there is a '' in polypeptides_chains
count = 0
for index, row in results_df.iterrows():
    type(row['polypeptides_chains'])
    for s in row['polypeptides_chains']:
        if s == '':
            count += 1

print(count)

TypeError: 'NoneType' object is not iterable